In [1]:
# armazenar os documentos totais em variavel

import os
rootdir = '/media/r2-d2/E0C494CDC494A6F6/doutorado/20_newsgroup'
docs = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #if os.path.basename(subdir) in ['alt.atheism', 'rec.sport.baseball']:
        with open(os.path.join(subdir, file), encoding='cp1252') as f:                
            contents = f.read()
            docs.append({ 'tema': os.path.basename(subdir), 'nome_arquivo': file, 'conteudo': contents  })

In [2]:
# baixando as libs para pre processamento

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

nltk.download('stopwords')

stopwords = set(stopwords.words('english'))
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to /home/r2-d2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# pre processamento

    # Remover caracteres especiais e números
    #text = re.sub('[^A-Za-z\s\']+', '', text)

def preprocess_text(text):
    # Converter tudo para letras minúsculas
    text = text.lower()
    
    # Remover as tags HTML
    text = re.sub('<[^<]+?>', '', text)
    

    
    # Tokenizar o texto em palavras
    words = nltk.word_tokenize(text)

    # remover palavras que so possua caractere especial
    words = [word for word in words if not re.match('^[^A-Za-z0-9]+$', word)]

    # remover palavras que tenha tamanho 1
    words = [word for word in words if len(word) > 1]
    
    # Remover as stopwords
    words = [word for word in words if word not in stopwords]
    
    # Aplicar o stemming
    words = [stemmer.stem(word) for word in words]
    
    # Juntar as palavras em uma única string
    text = ' '.join(words)

    
    return text



In [4]:
# rodar o pre processamento

def preprocess_collection(docs):

    preprocessed_documents = []

    for document in docs:
        tema = document['tema']
        nome_arquivo = document['nome_arquivo']
        conteudo = document['conteudo']
        
        preprocessed_conteudo = preprocess_text(conteudo)
        
        preprocessed_document = {
            'tema': tema,
            'nome_arquivo': nome_arquivo,
            'conteudo': preprocessed_conteudo
        }
        
        preprocessed_documents.append(preprocessed_document)
        
    return preprocessed_documents

In [ ]:
# Pré-cálculo do IDF para cada termo em todos os documentos
# LEGADO

# import math

# frases = [o['conteudo'] for o in preprocessed_documents]
# n = len(frases)
# idfs = {}
# for frase in frases:
#     for termo in set(frase.split()):
#         idfs[termo] = math.log(n / sum(1 for f in frases if termo in f))    

In [6]:
preprocessed_documents = preprocess_collection(docs)

In [7]:
frases = [o['conteudo'] for o in preprocessed_documents]

In [8]:
# Pré-cálculo do IDF para cada termo em todos os documentos


from collections import Counter
import math

n = len(frases)
termos_doc_count = Counter()

# Contagem do número de documentos que contém cada termo
for frase in frases:
    termos = set(frase.split())
    for termo in termos:
        termos_doc_count[termo] += 1

idfs = {}
# Cálculo dos valores IDF para cada termo
for termo, doc_count in termos_doc_count.items():
    idf = math.log(n / doc_count)
    idfs[termo] = idf

In [9]:
# Função para calcular o TF-IDF

def tfidf(termo, documento, idfs):
    tf = documento.count(termo)
    idf_valor = idfs.get(termo, 0)
    return tf * idf_valor

In [ ]:
# Função para gerar o ranking de similaridade

# LEGADO

# def similaridade(texto, documentos, idfs):
   
#     # separando as palavras do texto
#     texto = texto.split()
    
#     # Lista de documentos com os valores TF-IDF
#     documentos_tfidf = []
    
#     for doc in documentos:
#         # Normalização do documento

#         documento = doc['conteudo'].split()
        
#         # Cálculo do valor TF-IDF para cada termo do texto
#         tfidf_documento = sum(tfidf(termo, documento, idfs) for termo in texto)
        
#         # Adição do documento e seu valor TF-IDF na lista de documentos
#         documentos_tfidf.append((doc['nome_arquivo'], doc['tema'], doc['conteudo'], tfidf_documento))

    
#     # Ordenação dos documentos pela similaridade
#     documentos_tfidf = sorted(documentos_tfidf, key=lambda x: x[3], reverse=True)
    
#     # Retorno dos 10 documentos mais similares com suas respectivas similaridades
#     return documentos_tfidf[:20]

In [10]:
# NOVA FUNCAO

def similaridade(texto, documentos, idfs):
    # Separar as palavras do texto
    texto_palavras = texto.split()

    # Calcular o IDF para cada palavra do texto
    idf_texto = {termo: idfs[termo] for termo in texto_palavras}

    # Pré-processar os documentos
    documentos_preprocessados = []
    for doc in documentos:
        # Dividir o documento em palavras
        documento_palavras = doc['conteudo'].split()

        # Remover as palavras que não aparecem no texto
        documento_palavras = [word for word in documento_palavras if word in idf_texto]

        # Armazenar o documento pré-processado
        documentos_preprocessados.append((doc['nome_arquivo'], doc['tema'],  doc['conteudo'], documento_palavras))

    # Calcular o TF-IDF para cada documento
    documentos_tfidf = []
    for nome_arquivo, tema, conteudo, documento_palavras in documentos_preprocessados:
        tfidf_documento = sum(idf_texto[termo] * documento_palavras.count(termo) for termo in texto_palavras)
        documentos_tfidf.append((nome_arquivo, tema, conteudo, tfidf_documento))

    # Ordenar os documentos pela similaridade
    documentos_tfidf = sorted(documentos_tfidf, key=lambda x: x[3], reverse=True)

    # Retornar os 20 documentos mais similares com suas respectivas similaridades
    return documentos_tfidf[:100]


In [11]:
print(preprocessed_documents[0]['conteudo'])
print(preprocessed_documents[0]['tema'])
print(preprocessed_documents[0]['nome_arquivo'])


xref cantaloupe.srv.cs.cmu.edu talk.abortion:120737 alt.atheism:53341 talk.religion.misc:83726 path cantaloupe.srv.cs.cmu.edu magnesium.club.cc.cmu.edu news.sei.cmu.edu cis.ohio-state.edu pacific.mps.ohio-state.edu zaphod.mps.ohio-state.edu usc sol.ctr.columbia.edu ira.uka.d germany.eu.net thoth.mchp.sni.d horus.ap.mchp.sni.d d012s658 frank frank d012s658.uucp frank o'dwyer newsgroup talk.abort alt.ath talk.religion.misc subject 2000 year say christian moral date 20 apr 1993 20:00:08 gmt organ siemens-nixdorf ag line 26 message-id refer nntp-posting-host d012s658.ap.mchp.sni.d articl mathew write frank d012s658.uucp frank o'dwyer write articl mathew mantis.co.uk mathew write ask think wrong relativ correct misconcept well cut chase admit find least attract realtiv elev heinou level good say effect 's good n't good thu terrorist elev level man peac complet nonsens relativ mean say absolut standard moral mean say standard moral equal good presum mean moral system better other manag witho

In [ ]:
#ranking = similaridade(preprocessed_documents[0]['conteudo'], preprocessed_documents, idfs)
for doc in ranking:
    #print(f"Documento {doc[0]} - Target {doc[1]} -->  {doc[2]} - Similaridade: {doc[3]}")
    print(f"Documento {doc[0]} - Target {doc[1]} --> Similaridade: {doc[3]}")


In [16]:
# Exemplo de uso - NOVO

#ranking = similaridade(preprocessed_documents[17502]['conteudo'], preprocessed_documents, idfs)

#Impressão do ranking
#for i, doc in enumerate(ranking):
acertos = 0
#for doc in ranking:
for i, doc in enumerate(ranking):
    #print(f"Documento {doc[0]} - Target {doc[1]} -->  {doc[2]} - Similaridade: {doc[3]}")
    if doc[1] == 'alt.atheism':
        acertos+=1
    if i+1 == 10:
        print(f"10 primeiros acertou {acertos}")
        print(str(acertos/float(10)))
    if i+1 == 20:
        print(f"20 primeiros acertou {acertos}")
        print(str(acertos/float(20)))
    if i+1 == 50:
        print(f"50 primeiros acertou {acertos}")                
        print(str(acertos/float(50)))
    if i+1 == 100:
        print(f"100 primeiros acertou {acertos}")        
        print(str(acertos/float(100)))


10 primeiros acertou 2
0.2
20 primeiros acertou 5
0.25
50 primeiros acertou 16
0.32
100 primeiros acertou 37
0.37


In [24]:
# armazenar os documentos totais em variavel

import os
rootdir = '/media/r2-d2/E0C494CDC494A6F6/doutorado/mini_newsgroups'
docs_consultas = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        with open(os.path.join(subdir, file), encoding='cp1252') as f:                
            contents = f.read()
            docs_consultas.append({ 'tema': os.path.basename(subdir), 'nome_arquivo': file, 'conteudo': contents  })

In [49]:
dirs_abs = []
for subdir, dirs, files in os.walk(rootdir):
    if os.path.basename(subdir) != '20_newsgroup':
        dirs_abs.append(os.path.basename(subdir))

temas = {}
for d in dirs_abs:
    temas[d] = {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0}

print(temas)    

{'mini_newsgroups': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0}, 'alt.atheism': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0}, 'comp.graphics': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0}, 'comp.os.ms-windows.misc': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0}, 'comp.sys.ibm.pc.hardware': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0}, 'comp.sys.mac.hardware': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0}, 'comp.windows.x': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0}, 'misc.forsale': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0}, 'rec.autos': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0}, 'rec.motorcycles': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0}, 'rec.sport.baseball': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0}, 'rec.sport.hockey': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0}, 'sci.crypt': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0}, 'sci.electronics': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0}, 'sci.med': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0}, 'sci.space': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0}, 'soc.religion.ch

In [25]:
#processando

doc_consultas_preproc = preprocess_collection(docs_consultas)

In [50]:
from statistics import mean

from IPython.display import clear_output

# consultando
p10 = []
p20 = []
p50 = []
p100 = []
tema_atual = ""

for c, doc_consulta in enumerate(doc_consultas_preproc):
    clear_output(wait=True)
    print(c)
    ranking = similaridade(doc_consulta['conteudo'], preprocessed_documents, idfs)
    acertos = 0
    for i, doc in enumerate(ranking):
        #print(f"Documento {doc[0]} - Target {doc[1]} -->  {doc[2]} - Similaridade: {doc[3]}")
        if doc[1] == doc_consulta['tema']:
            acertos+=1
        if i+1 == 10:
            p10.append(acertos/10.0)            
        if i+1 == 20:
            p20.append(acertos/20.0)
        if i+1 == 50:
            p50.append(acertos/50.0)
        if i+1 == 100:
            p100.append(acertos/100.0) 
    
    if c > 0:
        if tema_atual != doc_consulta['tema'] or c == len(doc_consultas_preproc) - 1:
            temas[tema_atual]['p10'] = mean(p10)
            temas[tema_atual]['p20'] = mean(p20)
            temas[tema_atual]['p50'] = mean(p50)
            temas[tema_atual]['p100'] = mean(p100)
            p10 = []
            p20 = []
            p50 = []
            p100 = []

    tema_atual = doc_consulta['tema']
    # if c > 202:
    #     break    


1999


In [51]:
temas

{'mini_newsgroups': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0},
 'alt.atheism': {'p10': 0.2693069306930693,
  'p20': 0.22871287128712872,
  'p50': 0.20752475247524754,
  'p100': 0.22732673267326733},
 'comp.graphics': {'p10': 0.637, 'p20': 0.4815, 'p50': 0.2846, 'p100': 0.1994},
 'comp.os.ms-windows.misc': {'p10': 0.099,
  'p20': 0.0905,
  'p50': 0.1116,
  'p100': 0.1444},
 'comp.sys.ibm.pc.hardware': {'p10': 0.192,
  'p20': 0.148,
  'p50': 0.1186,
  'p100': 0.1296},
 'comp.sys.mac.hardware': {'p10': 0.254,
  'p20': 0.1785,
  'p50': 0.1084,
  'p100': 0.1008},
 'comp.windows.x': {'p10': 0.52, 'p20': 0.43, 'p50': 0.3282, 'p100': 0.2373},
 'misc.forsale': {'p10': 0.139,
  'p20': 0.10200000000000001,
  'p50': 0.0724,
  'p100': 0.072},
 'rec.autos': {'p10': 0.189, 'p20': 0.154, 'p50': 0.1334, 'p100': 0.1442},
 'rec.motorcycles': {'p10': 0.225, 'p20': 0.169, 'p50': 0.1332, 'p100': 0.132},
 'rec.sport.baseball': {'p10': 0.186,
  'p20': 0.154,
  'p50': 0.1386,
  'p100': 0.1521},
 'rec.sport.hoc

In [326]:
from statistics import mean

print(mean(p10))
print(mean(p20))
print(mean(p50))
print(mean(p100))

0.266
0.2255
0.2054
0.2262


In [52]:
import json
with open('data.json', 'w') as f:
    json.dump(temas, f)

In [53]:
import pandas as pd

df = pd.DataFrame(temas)
df = df.transpose()
df.drop('mini_newsgroups')

,p10,p20,p50,p100
alt.atheism,0.269307,0.228713,0.207525,0.227327
comp.graphics,0.637000,0.481500,0.284600,0.199400
comp.os.ms-windows.misc,0.099000,0.090500,0.111600,0.144400
comp.sys.ibm.pc.hardware,0.192000,0.148000,0.118600,0.129600
comp.sys.mac.hardware,0.254000,0.178500,0.108400,0.100800
comp.windows.x,0.520000,0.430000,0.328200,0.237300
misc.forsale,0.139000,0.102000,0.072400,0.072000
rec.autos,0.189000,0.154000,0.133400,0.144200
rec.motorcycles,0.225000,0.169000,0.133200,0.132000
rec.sport.baseball,0.186000,0.154000,0.138600,0.152100


In [54]:
df.describe()

,p10,p20,p50,p100
count,21.000000,21.000000,21.000000,21.000000
mean,0.286189,0.237988,0.195414,0.191916
std,0.160558,0.135536,0.107768,0.100052
min,0.000000,0.000000,0.000000,0.000000
25%,0.189000,0.154000,0.118600,0.132000
50%,0.254000,0.184000,0.163400,0.175600
75%,0.362000,0.303000,0.284600,0.257700
max,0.637000,0.539000,0.460000,0.451000
